In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



In [2]:
df= pd.read_csv(r'C:\Users\user\Downloads\0f411c708e55af442eafb33bfb7ee7585f5b0211a52d9ccc4287a23d8d6abe76_STOCK_INDEX.csv')

In [3]:
df_num = df.drop('Date',axis=1)

In [4]:
for i in df_num.columns:
    df_num[i].ffill(inplace=True)

In [5]:
# Create a new dataframe with only the 'Close column 
closing_prices = df_num.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = closing_prices.values

# Normalizing Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)


# Getting  the train and test array
training_data_len = int(( len(dataset) * .80 ))
train_data, test_data = scaled_data[:training_data_len], scaled_data[training_data_len:]


In [7]:
# Preparing Dataset
def prepare_x_train_y_train(data, stepping_time):
    x_train = []
    y_train = []
    for i in range(len(data)-stepping_time-1):
        x_train.append(data[i:i+stepping_time,0])
        y_train.append(data[i+stepping_time,0])
    return np.array(x_train), np.array(y_train)

In [8]:
stepping_time = 10
x_train, y_train = prepare_x_train_y_train(train_data, stepping_time)
x_test, y_test = prepare_x_train_y_train(test_data, stepping_time)

In [9]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(stepping_time,1)))
model.add(GRU(32,return_sequences=False))
model.add(Dropout(0.10))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=40,batch_size=32)

Epoch 1/40
67/67 [==============================] - 3s 13ms/step - loss: 0.0051 - val_loss: 6.0848e-04
Epoch 2/40
67/67 [==============================] - 0s 5ms/step - loss: 8.5442e-04 - val_loss: 5.6782e-04
Epoch 3/40
67/67 [==============================] - 0s 5ms/step - loss: 6.7289e-04 - val_loss: 5.9566e-04
Epoch 4/40
67/67 [==============================] - 0s 5ms/step - loss: 5.9672e-04 - val_loss: 5.3193e-04
Epoch 5/40
67/67 [==============================] - 0s 5ms/step - loss: 5.8084e-04 - val_loss: 5.7743e-04
Epoch 6/40
67/67 [==============================] - 0s 5ms/step - loss: 5.6747e-04 - val_loss: 5.1763e-04
Epoch 7/40
67/67 [==============================] - 0s 5ms/step - loss: 5.4557e-04 - val_loss: 7.8609e-04
Epoch 8/40
67/67 [==============================] - 0s 5ms/step - loss: 5.2070e-04 - val_loss: 5.4164e-04
Epoch 9/40
67/67 [==============================] - 0s 4ms/step - loss: 4.7038e-04 - val_loss: 4.7144e-04
Epoch 10/40
67/67 [==============================

In [2]:
# Normalizing Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

NameError: name 'MinMaxScaler' is not defined

In [10]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1,1))

# Get the root mean squared error (RMSE)
rmse = (np.sqrt(np.mean(((predictions - y_test) ** 2))))
rmse

17/17 [==============================] - 1s 2ms/step


170.74001841703708

In [11]:
# Save the model weights            
model.save_weights('MODEL3.h5')
print('GRU model weights saved successfully.')

GRU model weights saved successfully.
